In [ ]:
# !py -3 -m pip install pandas_gbq

In [ ]:
from google.cloud import bigquery
import google.auth
import io,os
import pandas as pd
import pandas_gbq

In [ ]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [ ]:
os.environ["HTTP_PROXY"] = "proxy.nas.medcity.net:80"
os.environ["HTTPS_PROXY"] = "proxy.nas.medcity.net:80"

In [ ]:
credentials, _ = google.auth.default()
credentials = google.auth.credentials.with_scopes_if_required(credentials, bigquery.Client.SCOPE)
authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

In [ ]:
bq_project_id = 'hca-hin-dev-cur-hr'
bq_dataset = 'edwhr_bi_views_copy'
src_bq_table = 'factopenings_daily'
bucket_name = "eim-cs-da-gmek-5764-dev"
folder_path = "tddump"

In [ ]:
source_query = "Select * From {}.{}.{}".format(bq_project_id, bq_dataset, src_bq_table)

# df = pd.read_gbq(source_query, project_id=bq_project_id)

bqclient = bigquery.Client(project=bq_project_id,credentials=credentials, _http=authed_http)
job_config = bigquery.job.LoadJobConfig()
# job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND
dataset_ref = bqclient.dataset(bq_dataset)
table_ref = dataset_ref.table(src_bq_table)
destination_uri = "gs://{}/{}/{}".format(bucket_name, folder_path, "factopenings_daily.csv")
job_config = bigquery.job.ExtractJobConfig()
# job_config.compression = bigquery.Compression.GZIP

extract_job = bqclient.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
    job_config=job_config,
)  # API request
extract_job.result()  # Waits for job to complete.